In [12]:
!pip install catboost xgboost openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 100.3 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2


In [16]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
import warnings
warnings.filterwarnings('ignore')

from sklearn.decomposition import PCA


A2. Use cross-validation techniques (RandomizedSearchCV()) technique to tune the
hyperparameters for your models.

In [9]:
# Load dataset
file_path = '/content/bert_features_output.xlsx'
df = pd.read_excel(file_path)


In [10]:
df_sample = df.iloc[:1000 , :]

In [11]:
# Features and Target
X = df_sample.drop(['label','text_'], axis=1)  # Change 'label' to your actual target column name
y = df_sample['label']



# Encode target if it's categorical
if y.dtype == 'object':
    le = LabelEncoder()
    y = le.fit_transform(y)

#Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define model and parameter grid
rf = RandomForestClassifier()

param_dist = {
    'n_estimators': [100, 200, 300, 400, 500],
    'max_depth': [None, 10, 20, 30, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

# Randomized Search
random_search = RandomizedSearchCV(
    rf, param_distributions=param_dist,
    n_iter=10, cv=3, verbose=2, random_state=42, n_jobs=-1
)

random_search.fit(X_train, y_train)

# Best Parameters and Evaluation
print("Best Parameters:", random_search.best_params_)
y_pred = random_search.predict(X_test)
print("Classification Report:\n", classification_report(y_test, y_pred))

Fitting 3 folds for each of 10 candidates, totalling 30 fits
Best Parameters: {'n_estimators': 400, 'min_samples_split': 10, 'min_samples_leaf': 4, 'max_depth': None, 'bootstrap': False}
Classification Report:
               precision    recall  f1-score   support

           0       0.66      0.70      0.68       104
           1       0.65      0.60      0.63        96

    accuracy                           0.66       200
   macro avg       0.65      0.65      0.65       200
weighted avg       0.65      0.66      0.65       200



A3. For projects dealing with classification problem, employ various other classifiers such as Support
Vector Machines, Decision Tree, RandomForest, CatBoost, AdaBoost, XGBoost, Naïve-Bayes & MLP.
Tabulate your results for your problem using different performance metrics. Your tabulated results
should compare between train and test results and make appropriate observations.

In [17]:

# Separate features and label
X = df.drop(columns=['label','text_'])  # assuming 'label' is the target column
y = df['label']

#REduce to 150 feautures
pca = PCA(n_components=150, random_state=42)
X = pca.fit_transform(X)
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


# Define models
models = {
    "SVM": SVC(),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(),
    "AdaBoost": AdaBoostClassifier(),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss'),
    "Naive Bayes": GaussianNB(),
    "MLP": MLPClassifier(max_iter=300)
}

# Evaluate models
results = []

for name, model in models.items():
    model.fit(X_train_scaled, y_train)
    y_train_pred = model.predict(X_train_scaled)
    y_test_pred = model.predict(X_test_scaled)

    results.append({
        "Model": name,
        "Train Accuracy": accuracy_score(y_train, y_train_pred),
        "Train Precision": precision_score(y_train, y_train_pred, average='weighted', zero_division=0),
        "Train Recall": recall_score(y_train, y_train_pred, average='weighted'),
        "Train F1": f1_score(y_train, y_train_pred, average='weighted'),
        "Test Accuracy": accuracy_score(y_test, y_test_pred),
        "Test Precision": precision_score(y_test, y_test_pred, average='weighted', zero_division=0),
        "Test Recall": recall_score(y_test, y_test_pred, average='weighted'),
        "Test F1": f1_score(y_test, y_test_pred, average='weighted')
    })

# Create and display results DataFrame
results_df = pd.DataFrame(results)
pd.set_option('display.max_columns', None)
print("\n📊 Classification Report Summary:\n")
print(results_df.sort_values(by="Test Accuracy", ascending=False))

# Optional: save to Excel
results_df.to_excel("model_comparison_results.xlsx", index=False)



📊 Classification Report Summary:

           Model  Train Accuracy  Train Precision  Train Recall  Train F1  \
0            SVM        0.941000         0.941094      0.941000  0.940998   
6            MLP        1.000000         1.000000      1.000000  1.000000   
4        XGBoost        1.000000         1.000000      1.000000  1.000000   
5    Naive Bayes        0.767625         0.768509      0.767625  0.767448   
2  Random Forest        1.000000         1.000000      1.000000  1.000000   
3       AdaBoost        0.741625         0.741842      0.741625  0.741558   
1  Decision Tree        1.000000         1.000000      1.000000  1.000000   

   Test Accuracy  Test Precision  Test Recall   Test F1  
0         0.8095        0.809521       0.8095  0.809486  
6         0.7850        0.785061       0.7850  0.784967  
4         0.7570        0.757032       0.7570  0.757004  
5         0.7420        0.742597       0.7420  0.741756  
2         0.7315        0.731511       0.7315  0.731475  


In [15]:
print(df.shape)

(10000, 770)
